<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/gif_resize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Paso 1: Instalar las bibliotecas necesarias
!pip install pillow imageio

# Paso 2: Importar las bibliotecas necesarias
from PIL import Image, ImageSequence
from google.colab import files
from IPython.display import display, Image as IPyImage
import io

# Función para redimensionar el GIF manteniendo la relación de aspecto
def resize_gif(input_gif, new_width):
    # Abrir el GIF original
    original_gif = Image.open(input_gif)

    # Calcular la nueva altura para mantener la relación de aspecto
    aspect_ratio = original_gif.height / original_gif.width
    new_height = int(new_width * aspect_ratio)

    frames = []
    durations = []
    disposals = []

    # Procesar cada cuadro del GIF
    for frame in ImageSequence.Iterator(original_gif):
        # Redimensionar el cuadro
        resized_frame = frame.convert('RGBA').resize((new_width, new_height), Image.ANTIALIAS)
        frames.append(resized_frame)

        # Obtener la duración de cada cuadro
        durations.append(frame.info.get('duration', 100))

        # Obtener la información de disposición de cada cuadro
        disposals.append(frame.disposal_method if 'disposal_method' in frame.info else 2)

    # Guardar el nuevo GIF en un buffer
    buffer = io.BytesIO()
    frames[0].save(
        buffer,
        format='GIF',
        save_all=True,
        append_images=frames[1:],
        loop=original_gif.info.get('loop', 0),
        duration=durations,
        disposal=disposals
    )
    buffer.seek(0)
    return buffer

# Función principal
def main():
    print("Por favor, sube el archivo GIF que deseas redimensionar.")

    # Paso 3: Cargar el GIF
    uploaded = files.upload()

    if not uploaded:
        print("No se ha cargado ningún archivo.")
        return

    gif_filename = next(iter(uploaded))
    print(f"Archivo cargado: {gif_filename}")

    # Mostrar el GIF original
    print("GIF Original:")
    display(IPyImage(gif_filename))

    # Solicitar el nuevo ancho al usuario
    while True:
        try:
            new_width = int(input("Ingrese el nuevo ancho deseado para el GIF (en píxeles): "))
            if new_width <= 0:
                print("Por favor, ingrese un número positivo.")
                continue
            break
        except ValueError:
            print("Entrada inválida. Por favor, ingrese un número entero.")

    # Redimensionar el GIF
    print("Redimensionando el GIF...")
    resized_gif_buffer = resize_gif(gif_filename, new_width)

    # Guardar el GIF redimensionado en un archivo temporal
    resized_gif_filename = f"resized_{gif_filename}"
    with open(resized_gif_filename, 'wb') as f:
        f.write(resized_gif_buffer.getbuffer())

    # Mostrar el GIF redimensionado
    print("GIF Redimensionado:")
    display(IPyImage(resized_gif_filename))

    # Paso 4: Descargar el GIF redimensionado
    print("Descargando el GIF redimensionado...")
    files.download(resized_gif_filename)

# Ejecutar la función principal
if __name__ == "__main__":
    main()